In [1]:
!pip install -U datasets angle-emb mlflow dagshub

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 215.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 195.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 172.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 226.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 244.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 272.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 241.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 279.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 237.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 277.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [1]:
import dagshub
dagshub.init(repo_owner='phamnguyentuong205', repo_name='aoe-tpp', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5dd8aa55-2fb3-4a97-8b98-4dd5fc0c5f63&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=261bf88578c618aab503a21d82c154ec98e5f81c3aa510f9ac4c832eca4f142b




Accessing as phamnguyentuong205

Initialized MLflow to track repo "phamnguyentuong205/aoe-tpp"

Repository phamnguyentuong205/aoe-tpp initialized!

In [2]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/phamnguyentuong205/aoe-tpp.mlflow")
mlflow.set_experiment("angle-triplet")


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in PromptModelConfig has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
2025/12/25 00:52:59 INFO mlflow.tracking.fluent: Experiment with name 'angle-triplet' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/46f72d147c464e0fb55886d25884a596', creation_time=1766623979823, experiment_id='1', last_update_time=1766623979823, lifecycle_stage='active', name='angle-triplet', tags={}>

In [3]:
from datasets import load_dataset
from angle_emb import AnglE
import torch
import math

# 1. CẤU HÌNH
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Thay bằng tên dataset của bạn trên Hugging Face

# 2. LOAD MODEL (PhoBERT cho tiếng Việt)
print("--> Đang tải model...")
angle = AnglE.from_pretrained(
    'vinai/phobert-base-v2',
    max_length=256,
    pooling_strategy='cls'
).to(device)


--> Đang tải model...


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [4]:
DATASET_NAME="anti-ai/ViNLI-SimCSE-supervised_v2"
print(f"Loading dataset '{DATASET_NAME}'")
dataset=load_dataset(DATASET_NAME,split="train")
print(len(dataset))

mapping = {
    'anchor': 'query',
    'pos': 'positive',
    'hard_neg': 'negative'
}

# Kiểm tra xem dataset có đủ cột không trước khi rename
print("Cột gốc:", dataset.column_names)

dataset = dataset.rename_columns(mapping)

print("Cột sau khi đổi:", dataset.column_names)
print("Mẫu dữ liệu:", dataset[0])



train_ds = dataset.select_columns(['query', 'positive', 'negative'])

print("--> Mẫu dữ liệu chuẩn bị train:", train_ds[0])
# Kiểm tra: Phải in ra đúng {'text': ..., 'positive': ..., 'negative': ...}

dataset_split = train_ds.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

Loading dataset 'anti-ai/ViNLI-SimCSE-supervised_v2'


README.md:   0%|          | 0.00/120 [00:00<?, ?B/s]

vinli_v2.jsonl:   0%|          | 0.00/78.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/275601 [00:00<?, ? examples/s]

275601
Cột gốc: ['anchor', 'pos', 'hard_neg']
Cột sau khi đổi: ['query', 'positive', 'negative']
Mẫu dữ liệu: {'query': 'Bạn biết trong mùa giải và tôi đoán ở cấp_độ của bạn uh bạn mất họ đến cấp_độ tiếp_theo nếu họ quyết_định để gọi lại đội_trưởng Braves quyết_định để gọi lại một người đàn_ông từ triple A sau đó một người đàn_ông đôi A đi lên để thay_thế anh ta và một người đàn_ông một đi lên để thay_thế anh ta', 'positive': 'Nếu mọi người nhớ lại , bạn sẽ mất mọi thứ lên cấp_độ tiếp_theo .', 'negative': 'Họ không bao_giờ gọi lại bất_cứ thứ gì .'}
--> Mẫu dữ liệu chuẩn bị train: {'query': 'Bạn biết trong mùa giải và tôi đoán ở cấp_độ của bạn uh bạn mất họ đến cấp_độ tiếp_theo nếu họ quyết_định để gọi lại đội_trưởng Braves quyết_định để gọi lại một người đàn_ông từ triple A sau đó một người đàn_ông đôi A đi lên để thay_thế anh ta và một người đàn_ông một đi lên để thay_thế anh ta', 'positive': 'Nếu mọi người nhớ lại , bạn sẽ mất mọi thứ lên cấp_độ tiếp_theo .', 'negative': 'Họ không ba

In [5]:
import torch
import gc

def clean_gpu_memory():
    # 1. Xóa các biến toàn cục thường gây tốn RAM (nếu có)
    # Lưu ý: Chỉ xóa các biến tạm, đừng xóa model hay loader nếu bạn còn cần dùng
    gc.collect()

    # 3. Xóa Cache PyTorch
    torch.cuda.empty_cache()

    # 4. (Tùy chọn) In thông số để kiểm tra
    print("GPU Memory Cleared!")
    print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"Reserved:  {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

# Gọi hàm
clean_gpu_memory()

GPU Memory Cleared!
Allocated: 0.50 GB
Reserved:  0.56 GB


In [6]:
import mlflow
import os

OUTPUT_DIR = "checkpoints/angle-phobert-triplet8h_2512"
BEST_MODEL_DIR = "checkpoints/best_model_final"

print("--> Bắt đầu training Triplet...")

with mlflow.start_run(run_name="angle-phobert-triplet-8h_2512"):

    # ====== LOG HYPERPARAMS ======
    mlflow.log_params({
        "batch_size": 256,
        "epochs": 5,
        "learning_rate": 2e-5,
        "warmup_steps": 200,
        "scheduler": "cosine",
        "loss": "angle + triplet",
        "cosine_w": 1.0,
        "ibn_w": 1.0,
        "angle_w": 0.02,
    })

    angle.fit(
        train_ds=train_dataset,
        valid_ds=eval_dataset,
        output_dir=OUTPUT_DIR,

        batch_size=256,
        epochs=5,
        learning_rate=2e-5,
        warmup_steps=200,
        gradient_accumulation_steps=1,

        fp16=True,
        bf16=False,

        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,

        argument_kwargs={
            "metric_for_best_model": "eval_loss",
            "greater_is_better": False,
            "logging_strategy": "epoch",
            "lr_scheduler_type": "cosine",
            "gradient_checkpointing": True,
        },

        loss_kwargs={
            "cosine_w": 1.0,
            "ibn_w": 1.0,
            "angle_w": 0.02,
            "cosine_tau": 20,
            "ibn_tau": 20,
            "angle_tau": 20,
        },
    )

    # ====== LƯU BEST MODEL ======
    angle.save_pretrained(BEST_MODEL_DIR)

    # ====== LOG ARTIFACTS (QUAN TRỌNG) ======
    mlflow.log_artifacts(OUTPUT_DIR, artifact_path="checkpoints")
    mlflow.log_artifacts(BEST_MODEL_DIR, artifact_path="best_model")

print(" Training + MLflow + DAGsHub hoàn tất")


2025/12/25 00:55:10 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



--> Bắt đầu training Triplet...


/home/admin/.local/lib/python3.10/site-packages/angle_emb/angle.py:336: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `AngleTrainer.__init__`. Use `processing_class` instead.
  super().__init__(**kwargs)


Epoch,Training Loss,Validation Loss
1,13.437300,3.502641
2,12.172400,3.339083
3,11.705700,3.239721
4,11.386700,3.271235
5,11.242800,3.288824


🏃 View run angle-phobert-triplet-8h_2512 at: https://dagshub.com/phamnguyentuong205/aoe-tpp.mlflow/#/experiments/1/runs/6732003fd8e444af973b91a1069a8d96
🧪 View experiment at: https://dagshub.com/phamnguyentuong205/aoe-tpp.mlflow/#/experiments/1
 Training + MLflow + DAGsHub hoàn tất


In [17]:
import mlflow
from angle_emb import AnglE
from datasets import load_dataset
import torch

# ======================================================
# 1. LOAD MODEL ĐÃ TRAIN (Round 1)
# ======================================================
PRETRAINED_MODEL_PATH = "checkpoints/best_model_final"
OUTPUT_DIR_ROUND_2 = "ckpts/sts-b-finetune-testv2" 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"--> Đang load model cũ từ: {PRETRAINED_MODEL_PATH}")

angle = AnglE.from_pretrained(
    PRETRAINED_MODEL_PATH,
    pooling_strategy='cls',
    is_llm=False
).to(device)

print(" Model đã load xong!")

# ======================================================
# 2. CHUẨN BỊ DỮ LIỆU (CHỈ CHIA TRAIN / VAL)
# ======================================================
DATASET_NAME_STS = "doanhieung/stsbenchmark-sts-vi"
print(f"Loading dataset '{DATASET_NAME_STS}'")

# Load toàn bộ dữ liệu
dataset = load_dataset(DATASET_NAME_STS, split="train")

# Chuẩn hóa format: text1, text2, label (0-1)
def process_data(obj):
    score = float(obj["score"])
    # Nếu điểm hệ 5 thì chia 5 để về hệ 0-1 (Quan trọng cho AnglE)
    if score > 1.0:
        score = score / 5.0
        
    return {
        "text1": str(obj["sentence1"]),
        "text2": str(obj["sentence2"]),
        "label": score
    }

dataset = dataset.map(process_data)
dataset = dataset.select_columns(["text1", "text2", "label"])

# --- CHIẾN THUẬT CHIA: 90% TRAIN - 10% VAL ---
# Vì dữ liệu ít nên ta dành phần lớn để train, chỉ chừa 1 ít để đánh giá lúc train
split_data = dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = split_data["train"]
eval_dataset = split_data["test"] # Ở đây "test" đóng vai trò là Validation

print("-" * 30)
print(f"Train size: {len(train_dataset)} mẫu")
print(f"Val size:   {len(eval_dataset)} mẫu")
print(f"Total:      {len(train_dataset) + len(eval_dataset)} mẫu")
print("-" * 30)

# ======================================================
# 3. TRAIN TIẾP (ROUND 2)
# ======================================================
print("--> Bắt đầu Training Round 2...")

with mlflow.start_run(run_name="STS-B_Finetune_TrainValOnlyv2"):
    
    mlflow.log_params({
        "dataset": "sts-vi",
        "split": "90/10",
        "base_model": "best_model_final"
    })

    angle.fit(
        train_ds=train_dataset,
        valid_ds=eval_dataset,
        output_dir=OUTPUT_DIR_ROUND_2,
        
        # --- Config Train ---
        batch_size=64,
        epochs=10,          # 10 Epoch là hợp lý cho STS
        learning_rate=1e-5, # LR an toàn
        
        # Bước lưu và đánh giá
        save_steps=100,
        eval_steps=100,
        logging_steps=100,
        warmup_steps=0,
        gradient_accumulation_steps=1,
        
        # Loss Config
        loss_kwargs={
            'cosine_w': 1.0,
            'ibn_w': 1.0,
            'angle_w': 0.02,
            'cosine_tau': 20,
            'ibn_tau': 20,
            'angle_tau': 20
        },
        
        # --- Quản lý Model ---
        fp16=True,
        save_total_limit=2,          # Chỉ giữ 2 bản backup
        
        argument_kwargs={
            "metric_for_best_model": "eval_loss",
            "greater_is_better": False,
            "report_to": "mlflow",
        }
    )

    # Lưu kết quả cuối cùng
    final_path = "checkpoints/best_model_sts_completev2"
    angle.save_pretrained(final_path)
    mlflow.log_artifacts(final_path, artifact_path="model_sts_completev2")

print(f"✅ Training hoàn tất! Model lưu tại: {final_path}")

--> Đang load model cũ từ: checkpoints/best_model_final
 Model đã load xong!
Loading dataset 'doanhieung/stsbenchmark-sts-vi'
------------------------------
Train size: 7765 mẫu
Val size:   863 mẫu
Total:      8628 mẫu
------------------------------
--> Bắt đầu Training Round 2...


/home/admin/.local/lib/python3.10/site-packages/angle_emb/angle.py:336: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `AngleTrainer.__init__`. Use `processing_class` instead.
  super().__init__(**kwargs)


Step,Training Loss,Validation Loss
100,8.249300,3.275276
200,7.713600,3.290809
300,7.643600,3.320662
400,7.587100,3.370884
500,7.520800,3.376744
600,7.491400,3.390934
700,7.410300,3.483723
800,7.376600,3.539976
900,7.354300,3.577469
1000,7.297700,3.561778


🏃 View run STS-B_Finetune_TrainValOnlyv2 at: https://dagshub.com/phamnguyentuong205/aoe-tpp.mlflow/#/experiments/1/runs/3c29435e31d7407585d31e381f1f2ed0
🧪 View experiment at: https://dagshub.com/phamnguyentuong205/aoe-tpp.mlflow/#/experiments/1
✅ Training hoàn tất! Model lưu tại: checkpoints/best_model_sts_completev2


In [18]:
from datasets import load_dataset

subsets = [
    "STS-B",
    "STS-Sickr",
    "STS12",
    "STS13",
    "STS14",
    "STS15"
]

def preprocess(ex):
    return {
        "text1": ex["sentence1"],
        "text2": ex["sentence2"],
        "label": ex["score"] / 5.0
    }

for subset in subsets:
    test_dataset = (
        load_dataset("anti-ai/ViSTS", subset, split="test")
        .map(preprocess)
        .select_columns(["text1", "text2", "label"])
    )

    corrcoef = angle.evaluate(test_dataset)
    print(f"{subset:10s} | Spearman: {corrcoef:.4f}")


Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

44it [00:01, 36.88it/s]                        


STS-B      | Spearman: 0.8167


Map:   0%|          | 0/9927 [00:00<?, ? examples/s]

311it [00:08, 38.14it/s]                         


...

118it [00:03, 36.73it/s]                         


STS14      | Spearman: 0.7130


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

94it [00:02, 36.86it/s]                        


STS15      | Spearman: 0.7865
